# Prepare the map of factorization of integers.

In [1]:
Mod4R3Map = {3:[(3,1)],7: [(7,1)]}
up = 1000000
for i in range(2, up):
    p = (i << 1)+1
    pList = list((p).factor())
    Mod4R3Map[p] = pList

# Section 1: Verification of the Lucas Test Formula

## Formulas of LPBP and $\mathcal{Z}$
Let $p, q$ be odd integers with $\left[\frac{-D}{p}\right]=\left[\frac{-D}{q}\right]=-1$, and $N=pq=\prod_i^s p_i^{r_i}$.

$e_4 =\frac{1}{4} \big(p+\left[\frac{-1}{p}\right]\big)\big(q+\left[\frac{-1}{q}\right]\big)$.

${\rm LPBP} = \prod_i \big(\gcd(e_4, p_i)-1\big) + \prod_i \gcd(e_4, p_i)$

Write $S = S_0 \cup S_1$, where $S_j:= \{i \mid r_i\equiv j \mod 2, 1 \leq i \leq s \}$. Then, one has, if $N$ is not a perfect square in $\mathbb{Z}$,
\begin{align}
    |\mathcal{Z}^{+1}(D,N)|
    = &\left[\frac{\prod_{i \in S} p_i^{r_i-1}}{2}\right]\left[\displaystyle\prod_{i\in S_0} \left(p_i-\left[ \frac{D}{p_i}\right]-1\right) \right]\notag\\ \cdot&\left[\prod_{i\in S_1} \left(p_i-\left[ \frac{D}{p_i}\right]-1\right) +(-1)^{|S_1|} \right]. \notag
\end{align}
Otherwise, if $N$ is a perfect square, $$|\mathcal{Z}^{+1}(D,N)|=\displaystyle\prod_{i\in S}  p_i^{r_i-1}\left(p_i-\left[\frac{D}{p_i}\right]-1\right).$$
Here $p_i-\left[\frac{D}{p_i}\right]=2^{k_i}d_i$ with  $2\nmid d_i$  for all $1 \leq i \leq s$.  

In [144]:
def lucasAllSetFormula(pList, D):
    result = 1
    innerPart = 1
    outerPart = 1
    count = 0

    isSquare = True
    for e in pList:
        exp = pList[e]
        if exp & 1 == 1:
            isSquare = False
            break
   
    if isSquare == True:
        for e in pList:
            exp = pList[e]
            prime = e
            result *=  prime^(exp-1) 
            qudraticDoverp = kronecker(D,prime)
            result *=(prime-qudraticDoverp-1)
            
        return result
    
    for e in pList:
        exp = pList[e]
        prime = e
        result *= prime^(exp-1)
        qudraticDoverp = kronecker(D,prime)
        if exp & 1 == 1:
            innerPart *= (prime-qudraticDoverp-1)
            count+=1
        else:
            outerPart *= (prime-qudraticDoverp-1)
    return (result*(innerPart+(-1)^count )*outerPart) >> 1

def lucasunitformula(allPList, p, q, e, mod, D):
    result = 1
    temp = 1
    for j in allPList:
        qudraticDoverp = kronecker(D,j)
        i = j-qudraticDoverp
        while i &1 != 1:
            i = i >>1
        gcddivisor = gcd(e,i)
        result  *= gcddivisor
        temp *= gcddivisor-1
    result += temp
    return result

## Basic Operation of quadratic order over $\mathcal{O}_D$

In [74]:
def Reduce(a,b,c,d, D, R):
    temp = R(c^2-d^2*D)^(-1)
    outputA = R(a*c-b*d*D)*temp
    outputB = R(b*c-a*d)*temp
    return outputA, outputB

In [75]:
def square(a,b, D,R):
    return R(a^2+b^2*D), R(2*a*b)

In [76]:
def mul(a1,b1, a2, b2,D,R):
    return R(a1*a2+b1*b2*D), R( a1*b2+b1*a2)

In [77]:
def Exp(a,b,D,R,exp):
    temp = "{0:b}".format(exp)
    resultA,resultB = 1,0
    powerA, powerB = a,b
    for i in range(len(temp)-1, -1, -1):
        bit = int(temp[i])
        if bit == 1:
            resultA, resultB = mul( resultA, resultB, powerA, powerB,D,R)
        powerA, powerB = square(powerA,powerB,D,R)
    return R(resultA), R(resultB)
            

## Calculation of LPBP and $\mathcal{Z}$ Based on Definitions

In [146]:
def GetCardinalityLucasTest(p, q, N, D):
    R = IntegerModRing(N)
    LPBP = []
    Z =  []
    minusR = N -1
    for i in range(0, N):
        temp = R(i^2 - D)
        if gcd(temp, N) != 1:
            continue
        Q = temp*R(4)^(-1)
        if kronecker(Q,N) == 1:
            Z.append(i)
        rA, rB = Reduce(i, 1, i, -1, D, R)
        resultA, resultB = Exp(rA,rB, D , R, int(e))
        if (resultA == 1 and resultB == 0) or (resultA == minusR and resultB == 0): 
            LPBP.append(i)
    return len(LPBP), len(Z)


## Verification of the Lucas Test Formula

In [185]:
LucasUp =100
for i in range(1, LucasUp):
    p = (i << 1)+1
    squartP = p^(0.5)
    # Remove square p
    if squartP*squartP == p*1.0:
        continue
        
    pList = Mod4R3Map[p]
    for j in range(1, LucasUp):
        if i < j:
            break
        q = (j << 1)+1
        # Remove square q
        squartQ = q^(0.5)
        if squartQ*squartQ == q*1.0:
            continue
        qList = Mod4R3Map[q]
        N = p*q
        # choose D such that [-D/p] = [-D/q] =-1.
        pMod4 = -1
        qMod4 = -1
        if p % 4 == 3:
            pMod4 = 1
        if q % 4 == 3:
            qMod4 = 1    
        D = 1
        for k in range(2, N):
            if gcd(N, k) != 1:
                continue
            if kronecker(k,p) == pMod4 and kronecker(k,q) == qMod4:
                D = k
                break
        e = (p-pMod4)*(q-qMod4)/4
        if gcd(e, N) != 1:
            continue
        primeList = {}
        for k in pList:
            primeList[k[0]] = k[1]
        for k in qList:
            if k[0] in primeList:
                primeList[k[0]] =  k[1]+primeList[k[0]]
                continue
            primeList[k[0]] = k[1]
        trueUnit, trueAllSet = GetCardinalityLucasTest(p,q,N,D)
        result = trueUnit/trueAllSet
        compare = lucasunitformula(primeList, p,q,e,N,D)/lucasAllSetFormula(primeList, D)*1.0
        if result != compare:
            print("Wrong-Formula:", p,q)

# Boneh-Franklin Test
## Formulas of BF and G
Let $p\equiv q \equiv 3 \mod 4$ with $\gcd(N,e_4)=1$. Here $e_4 = (p-1)(q-1)/4$.

Assume ${\rm BF}(N, e_4) := \left\{ g\in \mathbb{Z}_N^\times \ | \ g^{e_4} \equiv \pm 1 \pmod N\right\},$ and 
    $G(N) :=\left\{g\in \mathbb{Z}_N^\times \ \bigg | \ \left[ \frac{g}{N}\right] = 1\right\}$.
Then the formulas of both are given by
$$|{\rm BF}(N, e_4)|  =2\cdot \displaystyle\prod_{i=1}^s \gcd(e_4,d_i).$$
Here $p_i-1=2^{k_i}d_i$ with $2\nmid d_i$ for all $1 \leq i \leq s.$


If $N$ is square, then $|G(N)|=\phi(N)$; otherwise $|G(N)| = \phi(N)/2$.


In [153]:
def BonehAllSetFormula(primeList):
    result = 1
    isSquare = True
    for i in primeList:
        exp = primeList[i]
        p = i
        if exp & 1 == 1:
            isSquare = False
        result *= p^(exp-1)
        result *= (p-1)
    if isSquare == True:
        return result
    return result >> 1

def BonehUnitformula(primeList, e, mod):
    result = 1
    for j in primeList:
        i = j-1
        while i &1 != 1:
            i = i >> 1
        result  *= gcd(e, i) 
    return result << 1

## Calculation of BF and G Based on Definitions

In [154]:
def GetBonehFranklinTest(p,q,N):
    R = IntegerModRing(N)
    e = (N-p-q+1)/4
    if gcd(e, N) != 1:
        return "failure"
    
    Bh = []
    G =  []
    minusR = N -1
    for i in range(N):
        if kronecker(i,N) == 1:
            G.append(i)
            temp = R(i)^e
            if temp == 1 or temp == minusR:
                Bh.append(i)
    return len(Bh), len(G)

# Verification of the Boneh-Franklin Test Formula

In [155]:
# check the correctness of Boneh-Franklin test
BFUp =30
for i in range(BFUp):
    p = (i << 2)+3
    pList = Mod4R3Map[p]
    for j in range(BFUp):
        if i < j:
            break
        q = (j << 2)+3
        qList = Mod4R3Map[q]
        N = p*q
        e = (N-p-q+1) >> 2
        if gcd(N,e) > 1:
            continue

        primeList = {}
        for k in pList:
            primeList[k[0]] = k[1]
        for k in qList:
            if k[0] in primeList:
                primeList[k[0]] =  k[1]+primeList[k[0]]
                continue
            primeList[k[0]] = k[1]
        trueUnit, trueAllSet = GetBonehFranklinTest(p,q,N)
        result = trueUnit/trueAllSet
        compare = BonehUnitformula(primeList,e,N)/BonehAllSetFormula(primeList)
        if result != compare:
            print("Wrong-Formula:", p,q)

# Miller-Rabin Test
## Formulas of MR
Let $p = \displaystyle\prod_{i=1}^s p_i^{r_i}\equiv   3 \mod{4}$, and 
$${\rm MR}(p)  := \{g \in \mathbb{Z}_p^\times \mid g^{(p-1)/2} \equiv \pm 1 \mod p \}\subset \mathbb{Z}_p^\times.$$


Then $$|{\rm MR}\big(p \big)| = 2\prod_i^s \gcd\big( (p-1)/2, d_i\big).$$


In [158]:
def MillerRabinUintFormula(pList, p):
    result = 1
    e = (p-1) >> 1
    for j in pList:
        i = int(j[0])-1
        while i &1 != 1:
            i = i >> 1
        result  *= gcd(e, i)
    return result << 1

def MillerRabinAllSetFormula(pList):
    result = 1
    for i in pList:
        exp = i[1]
        p = i[0]
        result *= p^(exp-1)
        result *= (p-1)
    return result

## Calculation of MR and $\mathbb{Z}_p^\times$ Based on Definitions¶

In [159]:
def MillerRabinTest(p):
    R = IntegerModRing(p)
    minus1 = p-1
    e = minus1 >> 1
    L1 = []
    L2 = []
    for i in range(p):
        temp = R(i)^e
        if gcd(i, p) == 1:
            L2.append(i)
        if temp == 1 or  temp == minus1 == p-1:
            L1.append(i)
    return len(L1), len(L2), len(L1)/len(L2)

## Verification of the Miller-Rabin Test Formula

In [161]:
# check the coorrectness of Miller-Rabin
MillerRabinUp = 40
for i in range(0, MillerRabinUp):
    check = (i << 2)+3
    _, _, ratio = MillerRabinTest(check)
    if ratio != MillerRabinUintFormula(Mod4R3Map[check], check)/MillerRabinAllSetFormula(Mod4R3Map[check]):
        print(i, MillerRabinTest(check),MillerRabinUintFormula(Mod4R3Map[check], check)/MillerRabinAllSetFormula(Mod4R3Map[check]))

# Pairwise Comparison Among the Three Tests

In [165]:
P = Primes()
numberNondividePrime =  100

## Lucas test VS Boneh-Franklin Test

In [166]:
def getCompareBonehLucasTest(plow, pUp, qlow, qUp ):
    count = 0
    countBig1 = 0
    equalCount = 0
    countSmall1 = 0
    for i in range(plow,pUp):
        a = (i << 2)+3
        aListLength = len(Mod4R3Map[a])
        for j in range(qlow,qUp):
            if j>i:
                break
            b = (j << 2)+3
            bListLength = len(Mod4R3Map[b])
            if aListLength == 1 and bListLength == 1:
                continue
            N = a*b
            e = (N-a-b+1) >> 2
            if gcd(N, e) != 1:
                continue

            aList = Mod4R3Map[a]
            bList = Mod4R3Map[b]

            primeList = {}
            for k in aList:
                primeList[k[0]] = k[1]
            for k in bList:
                if k[0] in primeList:
                    primeList[k[0]] =  k[1]+primeList[k[0]]
                    continue
                primeList[k[0]] = k[1]
            
            isSkip = False
            for m in range(1, numberNondividePrime):
                prime = P[m]
                if prime in primeList:
                    isSkip = True
                    break
            if isSkip:
                continue
            
    
            buint = BonehUnitformula(primeList,e,N)
            bset = BonehAllSetFormula(primeList)
            Lunit = lucasunitformula(primeList, a,b,e,N)
            Lset = lucasAllSetFormula(primeList)
    
            boneh = 1.0*buint/bset
            lucas = 1.0*Lunit/Lset
            
            if boneh > lucas:
                countBig1 += 1
                continue
            elif boneh < lucas:
                countSmall1 += 1
                continue
            else:
                equalCount+=1
    print("equalCount:", equalCount)
    print("countBig1:", countBig1)
    print("countSmall1:", countSmall1)
            

## Lucas test VS Miller-Rabin test:

In [168]:
def getCompareMillerLucasTest(plow, pUp, qlow, qUp ):
    count = 0
    countBig1 = 0
    equalCount = 0
    countSmall1 = 0
    for i in range(plow,pUp):
        a = (i << 2)+3
        aListLength = len(Mod4R3Map[a])
        for j in range(qlow,qUp):
            if j>i:
                break
            b = (j << 2)+3
            bListLength = len(Mod4R3Map[b])
            if aListLength == 1 and bListLength == 1:
                continue
            N = a*b
            e = (N-a-b+1) >> 2
            if gcd(N, e) != 1:
                continue

            aList = Mod4R3Map[a]
            bList = Mod4R3Map[b]

            primeList = {}
            for k in aList:
                primeList[k[0]] = k[1]
            for k in bList:
                if k[0] in primeList:
                    primeList[k[0]] =  k[1]+primeList[k[0]]
                    continue
                primeList[k[0]] = k[1]
            
            isSkip = False
            for m in range(1, numberNondividePrime):
                prime = P[m]
                if prime in primeList:
                    isSkip = True
                    break
            if isSkip:
                continue
            Muint = MillerRabinUintFormula(aList,a)*MillerRabinUintFormula(bList,b)
            Mset = MillerRabinAllSetFormula(aList)*MillerRabinAllSetFormula(bList)
            Lunit = lucasunitformula(primeList, a,b,e,N)
            Lset = lucasAllSetFormula(primeList)
    
            miller = 1.0*Muint/Mset
            lucas = 1.0*Lunit/Lset
            
            if miller > lucas:
                countBig1 += 1
                continue
            elif miller < lucas:
                countSmall1 += 1
                continue
            else:
                equalCount+=1
    print("equalCount:", equalCount)
    print("countBig1:", countBig1)
    print("countSmall1:", countSmall1)
            

## Boneh-Franklin test VS Miller-Rabin test:¶

In [170]:
def getCompareBonehMillerTest(plow, pUp, qlow, qUp ):
    count = 0
    countBig1 = 0
    equalCount = 0
    countSmall1 = 0
    for i in range(plow,pUp):
        a = (i << 2)+3
        aListLength = len(Mod4R3Map[a])
        for j in range(qlow,qUp):
            if j>i:
                break
            b = (j << 2)+3
            bListLength = len(Mod4R3Map[b])
            if aListLength == 1 and bListLength == 1:
                continue
            N = a*b
            e = (N-a-b+1) >> 2
            if gcd(N, e) != 1:
                continue

            aList = Mod4R3Map[a]
            bList = Mod4R3Map[b]

            primeList = {}
            for k in aList:
                primeList[k[0]] = k[1]
            for k in bList:
                if k[0] in primeList:
                    primeList[k[0]] =  k[1]+primeList[k[0]]
                    continue
                primeList[k[0]] = k[1]
            
            isSkip = False
            for m in range(1, numberNondividePrime):
                prime = P[m]
                if prime in primeList:
                    isSkip = True
                    break
            if isSkip:
                continue
            
    
            buint = BonehUnitformula(primeList,e,N)
            bset = BonehAllSetFormula(primeList)
            Muint = MillerRabinUintFormula(aList,a)*MillerRabinUintFormula(bList,b)
            Mset = MillerRabinAllSetFormula(aList)*MillerRabinAllSetFormula(bList)
    
            boneh = 1.0*buint/bset
            miller = 1.0*Muint/Mset
            
            if boneh > miller:
                countBig1 += 1
                continue
            elif boneh < miller:
                countSmall1 += 1
                continue
            else:
                equalCount+=1
    print("equalCount:", equalCount)
    print("countBig1:", countBig1)
    print("countSmall1:", countSmall1)
            